In [1]:
import os    
os.environ['THEANO_FLAGS'] = "device=gpu"  
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from numpy import random
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import backend as K

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
from dgmm import loadtrainandlabel,loadtestandlabel
from lib.bdtb import simpanMSE, simpanMSEMiyawaki, plotDGMM,ubahkelistofchunks,simpanScore

import time

matlist=[]
matlist.append('data/de_s1_V1_Ecc1to11_baseByRestPre_smlr_s1071119ROI_resol10_leave0_1x1_preprocessed.mat')

In [2]:
matfile=matlist[0]
train_data,label=loadtrainandlabel(matfile)
testdt,testlb=loadtestandlabel(matfile)

In [3]:
predm,labelm,msem=simpanMSEMiyawaki()

data/s1_V1_Ecc1to11_baseByRestPre_smlr_s1071119ROI_resol10_figRecon_linComb-errFuncImageNonNegCon_1x1_maxProbLabel_dimNorm.mat


In [4]:
x=testlb.astype('float32')
y=testdt.astype('float32')
z=predm.astype('float32')

#split data train dan testing untuk keperluan random 
X_train, X_test, Y_train, Y_test, Miyawaki_1, Miyawaki_2 = train_test_split( x, y, z,test_size=20, random_state=0)

In [5]:
resolution = 10
#menambahkan channel di bagian belakang, jika tidak dipakai hapus saja angka 1 dibelakangnya
X_train = X_train.reshape([X_train.shape[0], resolution, resolution])
X_test = X_test.reshape([X_test.shape[0], resolution, resolution])


In [6]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))   
Y_train = min_max_scaler.fit_transform(Y_train)     
Y_test = min_max_scaler.transform(Y_test)

print ('X_train.shape : ')
print (X_train.shape)
print ('Y_train.shape')
print (Y_train.shape)
print ('X_test.shape')
print (X_test.shape)
print ('Y_test.shape')
print (Y_test.shape)
numTrn=X_train.shape[0]
numTest=X_test.shape[0]

X_train.shape : 
(100, 10, 10)
Y_train.shape
(100, 967)
X_test.shape
(20, 10, 10)
Y_test.shape
(20, 967)


In [8]:
filename=1
for stim in X_train:
  plt.imsave('./stim/'+str(filename)+'.png',np.rot90(np.fliplr(stim)))
  filename=filename+1

membuat model

In [9]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [10]:
def get_siamese_model(featurelength):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """

    # Define the tensors for the two input images
    left_input = Input(featurelength)
    right_input = Input(featurelength)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Dense(2048, input_dim=featurelength, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(100, activation='relu'))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(100,activation='relu',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [11]:
model = get_siamese_model(967)
model.summary()
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 967)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 967)]        0           []                               
                                                                                                  
 sequential (Sequential)        (None, 100)          5706340     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 100)          0           ['sequential[0][0]',         

In [12]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 4 #32 # jumlah data training
n_iter = 20000 # No. of training iterations
N_way = 2 #20 # how many classes for testing one-shot tasks, jumlah data testing 
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [14]:
model_path = './weights/'

berikutnya adalah kelompokkan input sinyal fmri dengan target yang mirip, karena inputan 2 fmri dengan outputan 1 label stimulus yang sama. kita kelompokkan dulu ya ges. bikinan sendiri

In [18]:
def get_batch(batch_size,s="train"):
      e_list=[11,14,18,20,30,37,46,51,60,65,67,78]#70
      Y_train_e = []
      X_train_e = []
      for e in e_list :
        Y_train_e.append(Y_train[e-1])
        X_train_e.append(X_train[e-1])

      Y_train_e = np.array(Y_train_e)
      X_train_e = np.array(X_train_e)


      Y_train_e1,Y_train_e2=np.array_split(Y_train_e, 2)
      #X_train_e1,X_train_e2=np.array.split(X_train_e,2)
      X_train_e[:5]
      pairs = Y_train_e1[:batch_size],Y_train_e2[:batch_size]
      targets = X_train_e[:batch_size]
      return pairs, targets

bikinan sendiri modif

In [19]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. bikin untuk genrate data testing"""
    e_list=[11,14,18,20,30,37,46,51,60,65,67,78]#70
    Y_train_e = []
    X_train_e = []
    for e in e_list :
      Y_train_e.append(Y_train[e-1])
      X_train_e.append(X_train[e-1])

    Y_train_e = np.array(Y_train_e)
    X_train_e = np.array(X_train_e)

    Y_train_e1,Y_train_e2=np.array_split(Y_train_e, 2)
    #X_train_e1,X_train_e2=np.array.split(X_train_e,2)
    if s == 'train':
        pairs = Y_train_e1[:batch_size],Y_train_e2[:batch_size]
        targets = X_train_e[:batch_size]
    else:
        pairs = Y_train_e1[-N:],Y_train_e2[-N:]
        targets = X_train_e[-N:]
    return pairs, targets

def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct